In [3]:
import numpy as np
import pandas as pd
import os

# File paths
technical_coeff_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/technical_coefficients/technical_coefficients_2019.csv"
price_volatility_path = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/price_data/II_PI_volatility.csv"
output_dir = "C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results"

# CPI weight file paths
individual_weights_file = os.path.join(output_dir, "individual_cpi_weights.csv")
eu28_weights_file = os.path.join(output_dir, "eu28_and_row_cpi_weights.csv")

# Load data
unweighted_impacts = pd.read_csv(os.path.join(output_dir, "unweighted_shock_impacts.csv"), index_col=0)
technical_coefficients = pd.read_csv(technical_coeff_path, index_col=0)
price_volatility = pd.read_csv(price_volatility_path, index_col=0)


# Function for individual weights
def calculate_individual_impacts(cpi_weights):
    impacts = []
    for exogenous_sector in unweighted_impacts.columns:
        # Clean sector code
        if exogenous_sector.startswith('ip_'):
            exogenous_sector = exogenous_sector[3:]
        
        # Extract country and sector codes
        try:
            country_code, sector_code = exogenous_sector.split('_', 1)
        except ValueError:
            print(f"Unexpected format for sector label: {exogenous_sector}")
            continue

        # Retrieve price shock
        try:
            price_shock = price_volatility.loc[exogenous_sector, 'price_volatility']
        except KeyError:
            print(f"Price shock not found for {exogenous_sector}. Skipping.")
            continue

        # Retrieve CPI weight for individual countries
        try:
            cpi_weight = cpi_weights.loc[exogenous_sector, country_code]
        except KeyError:
            print(f"CPI weight not found for {exogenous_sector} in {country_code}. Skipping.")
            continue

        # Calculate direct impact
        direct_impact = cpi_weight * price_shock

        # Calculate indirect impacts
        try:
            A_EE = technical_coefficients.drop(index=[f"op_{exogenous_sector}"], errors='ignore').drop(columns=[f"ip_{exogenous_sector}"], errors='ignore')
            propagated_shocks = A_EE.loc[:, exogenous_sector] * price_shock
            indirect_impact = (
                (cpi_weights.loc[propagated_shocks.index, country_code] * propagated_shocks)
                .drop(index=exogenous_sector, errors='ignore')
                .sum()
            )
        except KeyError as e:
            print(f"Error during indirect impact calculation for {exogenous_sector}: {e}")
            indirect_impact = 0

        # Calculate total impact
        total_impact = direct_impact + indirect_impact

        # Store results
        impacts.append({
            'Country': country_code,
            'Sector': sector_code,
            'Direct Impact': direct_impact,
            'Indirect Impact': indirect_impact,
            'Total Impact': total_impact
        })

    # Save results
    impacts_df = pd.DataFrame(impacts)
    output_file = os.path.join(output_dir, "individual_impacts.csv")
    impacts_df.to_csv(output_file, index=False)
    print(f"Individual impacts saved to {output_file}")


# Function for EU28 weights
def calculate_eu28_impacts(cpi_weights):
    impacts = []
    for exogenous_sector in unweighted_impacts.columns:
        # Clean sector code
        if exogenous_sector.startswith('ip_'):
            exogenous_sector = exogenous_sector[3:]

        # Retrieve price shock
        try:
            price_shock = price_volatility.loc[exogenous_sector, 'price_volatility']
        except KeyError:
            print(f"Price shock not found for {exogenous_sector}. Skipping.")
            continue

        # Retrieve CPI weight for EU28
        try:
            cpi_weight = cpi_weights.loc[exogenous_sector, 'EU28']
        except KeyError:
            print(f"CPI weight not found for {exogenous_sector}. Skipping.")
            continue

        # Calculate direct impact
        direct_impact = cpi_weight * price_shock

        # Calculate indirect impacts
        try:
            A_EE = technical_coefficients.drop(index=[f"op_{exogenous_sector}"], errors='ignore').drop(columns=[f"ip_{exogenous_sector}"], errors='ignore')
            propagated_shocks = A_EE.loc[:, exogenous_sector] * price_shock
            indirect_impact = (
                (cpi_weights.loc[propagated_shocks.index, 'EU28'] * propagated_shocks)
                .drop(index=exogenous_sector, errors='ignore')
                .sum()
            )
        except KeyError as e:
            print(f"Error during indirect impact calculation for {exogenous_sector}: {e}")
            indirect_impact = 0

        # Calculate total impact
        total_impact = direct_impact + indirect_impact

        # Store results
        impacts.append({
            'Sector': exogenous_sector,
            'Direct Impact': direct_impact,
            'Indirect Impact': indirect_impact,
            'Total Impact': total_impact
        })

    # Save results
    impacts_df = pd.DataFrame(impacts)
    output_file = os.path.join(output_dir, "eu28_impacts.csv")
    impacts_df.to_csv(output_file, index=False)
    print(f"EU28 impacts saved to {output_file}")


# Load and process individual weights
individual_cpi_weights = pd.read_csv(individual_weights_file, index_col=0)
individual_cpi_weights.index = individual_cpi_weights.index.str.replace('^op_', '', regex=True)
calculate_individual_impacts(individual_cpi_weights)

# Load and process EU28 weights
eu28_cpi_weights = pd.read_csv(eu28_weights_file, index_col=0)
eu28_cpi_weights.index = eu28_cpi_weights.index.str.replace('^op_', '', regex=True)
calculate_eu28_impacts(eu28_cpi_weights)


Individual impacts saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\individual_impacts.csv
EU28 impacts saved to C:/Users/danie/Nextcloud/Coding/Masterthesis/data/processed/results\eu28_impacts.csv
